# Anisotropic constant pressure MD¶

## Summary

This exercise studies a well-known phase transition in potassium chloride, see ref. (Parrinello and A. Rahman, Polymorphic transitions in alkali halides. A molecular dynamics study, Journal de Physique Colloques, 42 C6, p. C6, 1981, doi: 10.1051/jphyscol:19816149, URL https://hal.archives-ouvertes.fr/jpa-00221214.), using constant pressure molecular dynamics. The objective is to develop the best practice in using such algorithms and to learn how phase transitions can be induced, detected and monitored in a simulation.

## Background

Potassium chloride at ambient temperature and pressure adopts the cubic rocksalt structure, in which each ion is surrounded by six ions of opposite charge in an octahedral arrangement. Under high pressure this structure transforms to something more close packed - the so-called caesium chloride structure, where the nearest neighbour coordination rises to eight ions. (Using the model potential adopted here, this occurs at about 1.4 GPa.) In this exercise the student will have the opportunity to see this phase transition using the method of anisotropic constant pressure molecular dynamics. Commencing with the rocksalt crystal structure and applying a fixed external pressure it is possible to induce the phase transition in a simulation. Similarly it is possible to see the reverse transition back to rocksalt. However it is not necessarily trivial to make these transitions happen in any given simulation (though you may be lucky the first time!) Your task will be to find the conditions under which the phase transition occurs. This will not be entirely a matter of finding the right conditions of temperature and pressure, but will also involve setting up the control parameters for the simulation so as to encourage the phase transition to occur. (Even if the transformation is thermodynamically permitted, it does not follow that it will happen in the lifetime of a simulation.)



## Setup

Be sure you have setup your DLPOLY environment by executing the appropiate setup environment. eg



In [ ]:
%%bash

git clone --depth=1 https://gitlab.com/ccp5/dl-poly.git

cmake -S dl-poly -Bbuild-dlpoly -DCMAKE_BUILD_TYPE=Release  && cmake --build build-dlpoly  
cd build-dlpoly/
make install
cd ../
pip install dlpoly-py ase matplotlib py3Dmol ipywidgets

Now we will need the input files for DLPOLY, a control file, a FIELD file and a CONFIG file. The last of these is a crystal of potassium chloride at ambient temperature and pressure (i.e. in the rocksalt structure). You should proceed as follows:

In [ ]:
%%bash

mkdir -p Exercise1
cd Exercise1
wget https://gitlab.com/drFaustroll/dlpoly-py/-/raw/devel/examples/data/ex1/CONFIG
wget https://gitlab.com/drFaustroll/dlpoly-py/-/raw/devel/examples/data/ex1/FIELD


now we will create the control files in this case is called ex1.control and saved in folder Exercise1/

In [ ]:
%%writefile Exercise1/ex1.control
title DL_POLY: Potassium Chloride Test Case 

temperature  300.0 K
print_frequency  10 steps
stats_frequency  10 steps
rdf_print ON
#rdf_calculate ON
rdf_frequency  10 steps
vdw_cutoff  7.0 ang
padding  0.25 ang
cutoff  7.0 ang
coul_method ewald
ewald_precision 1e-06
ensemble nst
ensemble_method hoover
ensemble_thermostat_coupling  0.1 ps
ensemble_barostat_coupling  1.0 ps
time_run  2000 steps
time_equilibration  500 steps
time_job  3600.0 s
time_close  100.0 s
timestep  0.001 ps
pressure_hydrostatic  40.0 katm
restart continue
rescale_frequency  10 steps
traj_calculate ON
traj_start  500 steps
traj_interval  100 steps
traj_key pos-vel


also we will add soem helper function showrdf to allow us to visualise the rdf

In [ ]:
from dlpoly import DLPoly
from dlpoly.rdf import rdf
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
def showrdf(loc):
    m = rdf(loc)
    for i in range(len(m.labels)):
        plt.plot(m.x, m.data[i,:,0],label = "-".join(m.labels[i]))
    plt.xlabel("r [Å])")
    plt.ylabel("gofr [a.u.])")
    plt.legend()

load the control, field and config and setup a working directory called w40 in this case.
second line is running dlpoly for you and the third line will show your the rdf.
Last three lines indicate at what pressure and temperature the simulation is run (check the manual for the units) and the statistical ensemble we use to sample.

In [ ]:
dlPoly = DLPoly(control="Exercise1/ex1.control", config="Exercise1/CONFIG",
                field="Exercise1/FIELD", workdir="w40")
dlPoly.run(numProcs = 1)
showrdf("w40/RDFDAT")
print("Pressure: {} unit".format(*dlPoly.control.pres))
print("Temperature: {} unit".format(dlPoly.control.temp))
print(dlPoly.control.ensemble)

now we will visualize REVCON the last frame of your simulation

In [ ]:
from ase.io import read,write
import py3Dmol

styles = ['line', 'stick', 'sphere']

def atomsView(a, size=(300, 300), style="stick",background='0xfeeeee'):

    assert style in styles

    write("temp.cif",a)
    c = open("temp.cif",'r').read()
    view = py3Dmol.view()
    view.addModel(c,'cif')
    view.setStyle({style:{}})
    view.setBackgroundColor(background)
    view.zoomTo()
    view.addUnitCell()

    return view

a=read("w40/REVCON",format="dlp4")
a.wrap()

atomsView(a)

now inspect the text output of your simulation, (replace cat with your favourite graphical text editor if you run on a local jupyter server)

In [ ]:
!cat w40/OUTPUT

Repeat the simulation at a different state point, where you might expect a phase transition to occur. Examine the result graphically once again (using the REVCON file) and try to deduce how the phase transition occurred. Look at the RDF plots and try to determine what phase the structure is now in.

as previously you load the files and setup a new working directory.
change the pressure and temperature to the desired values (60 for pressure, 500 for temperature) and rerun

In [ ]:
dlPoly = DLPoly(control="Exercise1/ex1.control", config="Exercise1/CONFIG",
                field="Exercise1/FIELD", workdir="w60")
dlPoly.control.pres = 60
dlPoly.control.temp = 500
dlPoly.run(numProcs = 1)
showrdf("w60/RDFDAT")
a=read("w60/REVCON",format="dlp4")
a.wrap()

atomsView(a)

If you do not see a phase transition, experiment with the control parameters (e.g. change the relaxation times, temperature or pressure, as you think fit) until you see one. Be as systematic as you can, using whatever insight you gain to rationalise what’s going on.

In [ ]:
dlPoly = DLPoly(control="Exercise1/ex1.control", config="Exercise1/CONFIG",
                field="Exercise1/FIELD", workdir="w50")
dlPoly.control.pres = 50
dlPoly.control.temp = 500
dlPoly.control.ensemble.args=(0.1,1.0)
dlPoly.run(numProcs = 1)
showrdf("w50/RDFDAT")
a=read("w50/REVCON",format="dlp4")
a.wrap()

atomsView(a)